# Exercício 2 - Enunciado

Considere-se de novo o algoritmo estendido de Euclides apresentado no TP2  mas usando o tipo dos inteiros e um parâmetro $N>0$
```
    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t
```

Este exercício é dirigido à prova de correção do algoritmo estendido de Euclides

1. Construa a asserção lógica que representa a pós-condição do algoritmo. Note que a definição da função  $\gcd$  é   $\gcd(a,b)\;\equiv\; \min \{\,r > 0\,|\,\exists\,s,t\,\centerdot\, r = a*s+b*t\,\}$ .
2. Usando a metodologia do comando havoc para o ciclo, escreva o programa na linguagem dos comandos anotados (LPA). Codifique a pós-condição do algoritmo com um comando assert .
3. Construa codificações do programa LPA através de transformadores de predicados: “weakest pre-condition” e “strongest post-condition”. 
4. Prove a correção  do programa LPA em ambas as codificações.

# Exercício 2 - Solução

```python
assert r = gcd(a,b) and r_prime = 0 and gcd(a,b) = a * s + b * t
```

In [1]:
from pysmt.shortcuts import *
from pysmt.typing import *

def prove(f):
    with Solver(name="z3") as s:
        s.add_assertion(Not(f))
        if s.solve():
            print("Failed to prove.")
        else:
            print("Proved.")

$$
\mathsf{Comando}\mathbin{\;::=\;} \mathit{var}\gets \mathit{exp}\;|\;\mathsf{havoc}\,\mathit{var}\;|\;\mathsf{assert}\,\varphi\;|\;\mathsf{assume}\,\phi
$$
$$
\mathsf{Fluxo}\mathbin{\;::=\;} \mathsf{skip}\;|\;\mathsf{Comando}\,;\,\mathsf{Fluxo}\;|\;\mathsf{Fluxo}\,\|\,\mathsf{Fluxo}
$$

### Weakest pre-condition

A denotação `[C]` associa a cada fluxo `C` um predicado que caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica WPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = True \\
[{\sf assert}\:\phi] = \phi \\
[ x = e ] = True \\
[(C_1 || C_2)] = [C_1] \wedge [C_2] \\
\\
[{\sf skip}\, ; C] = [C] \\
[{\sf assume}\:\phi\, ; C] = \phi \to [C] \\
[{\sf assert}\:\phi\, ; C] = \phi \wedge [C] \\
[ x = e \, ; C] = [C][e/x] \\
[(C_1 || C_2)\, ; C] = [(C_1 ; C) || (C_2 ; C)]
\end{array}
$


# Grammar

"""
S := CODE $
CODE := EXPRS | CYCLES | PRECOND | POSTCOND | IFS | SKIP | | 
CYCLE := while CONDS do CODE CODE
IFS := if CONDS then CODE IF CODE
IF := else CODE | 
PRECOND := assume CONDS CODE
POSTCOND := assert CONDS CODE
CONDS := id sinal id COND CODE
COND := and CONDS | or CONDS | 
EXPRS := ATRIBS EXPR CODE
ATRIBS := id ATRIB atrib
ATRIB := virg id ATRIB | 
EXPRE := EXP | EXP , EXPRE
EXP := TERM EXP_OPS
EXP_OPS := op_a TERM EXP_OPS | 
TERM := FACTOR TERM_OPS
TERM_OPS := op_m FACTOR TERM_OPS | 
FACTOR := id | int | lparen EXPR rparen
SKIP := skip CODE
"""

# TOKENS
"""
op_a = ["+","-"]
op_m = ["*", "div"]
id = r"[a-zA-z]+['_]*[a-zA-z]*"
atrib = ":="
int = r'\d+'
sinal = [">=", ">", "<", "<=", "=", "!="]
lparen, rparen = "(", ")"
while = "while"
if = "if"
then = "then"
do = "do"
else = "else"
skip = "skip"
assume = "assume"
assert = "assert"
"""

def tokenizer(str):
    op_a = [r"+",r"-"]
    op_m = [r"*", r"div"]
    id = r"[a-zA-z]+['_]*[a-zA-z]*"
    atrib = r":="
    int = r'\d+'
    sinal = [r">=", r">", r"<", r"<=", r"=", r"!="]
    lparen, rparen = r"(", r")"
    while = r"while"
    if = r"if"
    then = r"then"
    do = r"do"
    else = r"else"
    skip = r"skip"
    assume = r"assume"
    assert = r"assert"
    if str in op_a:
        return 0;
    if str in op_m:
        return 1;

```
    INPUT  a, b : Int
    assume  a > 0 and b > 0 and a < N and b < N
    r, r', s, s', t, t' = a, b, 1, 0, 0, 1
    while r' != 0
      q = r div r'
      r, r', s, s', t, t' = r', r − q × r', s', s − q × s', t', t − q × t' 
    OUTPUT r, s, t
```

``` python

Programa de fluxos.

Sejam pre = a > 0 and b > 0 and a < N and b < N
      inv = (r > 0) and (r < N) and (r = a*s + b*t)
      pos = r == gcd(a,b) and r_prime == 0 and gcd(a,b) == a * s + b * t
        
assume pre;
r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;
assert inv;
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';
((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)
assert pos;
```

Denotação lógica com WPC.
```python
[
assume a > 0 and b > 0 and a < N and b < N;
r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1;
assert inv;
havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t';
((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)
assert r = gcd(a,b) and r_prime = 0 and gcd(a,b) = a * s + b * t
]
=
pre -> [ assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; ...; assert pos ] [a/r][b/r'][1/s][0/s'][0/t][1/t']
=
pre -> inv[a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t] and (forall r,r',s,s',t,t'. [...; assert pos]) [a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t]
=
pre -> inv[a/r][b/r'][1/s][0/s'][0/t][1/t'][rP<-r][sP<-s][tP<-t] and
       (forall r,r',s,s',t,t'.
           ( (r' != 0 and inv) -> inv[r/rP][s/sP][t/tP][(tP-q*t')/t'][t'/t][(sP-q*s')/s'][s'/s][(rP-q*r')/r'][r'/r][(r/r')/q]) )
             and
             ( (not(r' != 0) and inv) -> pos)
       )
```

In [19]:
gcd = Symbol('gcd', FunctionType(INT,[INT,INT]))
aux = Symbol('aux', INT)
r = Symbol('r', INT)
r_prime = Symbol('r_prime', INT)
s = Symbol('s', INT)
s_prime = Symbol('s_prime', INT)
t = Symbol('t', INT)
t_prime = Symbol('t_prime', INT)


axioms = ForAll([a, b], Ite(Equals(b, Int(0)), Equals(gcd(a, b), a), Equals(gcd(a, b), gcd(b , Minus(a, Times(Div(a, b), b))))))

#ax1 = Implies(Equals(b,Int(0)),Equals(gcd(a,b),a))
#ax2 = Implies(GT(b, Int(0)), Equals(gcd(a, b), gcd(b,Minus(a, Times(Div(a, b), b)))))
#axioms = And(ax1, ax2)

In [21]:
prove(Implies(axioms,Equals(gcd(Int(12),Int(5)),Int(5))))

SolverReturnedUnknownResultError: 

In [22]:
N = Symbol('N', INT)
q = Symbol('q', INT)

pre = And(GT(a, Int(0)), GT(b, Int(0)), GT(N, a), GT(N, b))

def contraexemplo(a, b, r):
    r_P = Symbol('r_P', INT)
    s_P = Symbol('s_P', INT)
    t_P = Symbol('t_P', INT)
    return Implies(And(GT(r_P, Int(0)), LT(r_P, r)), inv(a, b, r_P, s_P, t_P))

def inv(a, b, r, s, t):
    return And(GT(r, Int(0)), GT(N, r), Equals(r, Plus(Times(a, s), Times(b, t))))

pos = And(inv(a, b, r, s, t), Not(contraexemplo(a, b, r)))

ini = substitute(inv(a, b, r, s, t), {r:a, r_prime:b, s:Int(1), s_prime:Int(0), t:Int(0), t_prime:Int(1)})
pres = Implies(And(Not(Equals(r_prime, Int(0))), inv(a, b, r, s, t)), 
               substitute(substitute(inv(a, b, r, s, t), 
                                     {r: r_prime, 
                                      r_prime: Minus(r, Times(q, r_prime)), 
                                      s: s_prime,
                                      s_prime: Minus(s, Times(q, s_prime)),
                                      t: t_prime,
                                      t_prime: Minus(t, Times(q, t_prime))}
                                    ), 
                          {q: Div(r, r_prime)}
                         )
              )

util = Implies(And(Equals(r_prime, Int(0)), inv(a, b, r, s, t)), pos)
vc = Implies(pre, And(ini, ForAll([r,r_prime,s,s_prime,t,t_prime], And(pres, util))))
prove(Implies(axioms, vc))

SolverReturnedUnknownResultError: 

### Strongest post-condition

Na abordagem SPC a denotação de um fluxo com um comando de atribuição introduz um quantificador existencial, o que não é adequado à verificação com SMT solvers: 
$ \quad [ C \; ; x = e ] \; =  \; \exists a. (x = e[a/x]) \wedge [C][a/x] $

Para lidar com este problema pode-se converter o programa original ao formato "*single assignment*" (SA).
Num programa SA cada variável só pode ser usada depois de ser atribuida e só pode ser atribuída uma única vez.

Um programa (onde variáveis são atribuídas mais do que uma vez) pode ser reescrito num programa SA criando "clones" distintos das variáveis de forma a que seja possível fazer uma atribuição única a cada instância.

Neste caso, a denotação `[C]` associa a cada fluxo `C` um predicado que caracteriza a sua correcção em termos lógicos (a sua VC) segundo a técnica SPC, sendo calculada pelas seguintes regras.

$
\begin{array}{l}
[{\sf skip}] = True \\
[{\sf assume}\:\phi] = \phi \\
[{\sf assert}\:\phi] = \phi \\
[x = e ] = (x = e)\\
[(C_1 || C_2)] = [C_1] \vee [C_2] \\
\\
[C \, ; {\sf skip}\;] = [C] \\
[C \, ;{\sf assume}\:\phi] = [C] \wedge \phi \\
[C \, ;{\sf assert}\:\phi] = [C] \to \phi \\
[ C \, ; x = e ] = [C] \wedge (x = e)\\
[C\,; (C_1 || C_2)] = [(C ; C_1) || (C; C_2)]
\end{array}
$



Denotação lógica com SPC
```python
[assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; ((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)
assert pos;
]
= 
[assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; ((assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False) || assume(r' == 0) and inv)] -> pos
=
[(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; (assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False)) || (assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; assume(r' == 0) and inv)] -> pos
=
([(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; (assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv; assume False))] or [(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; assume(r' == 0) and inv)]) -> pos
=
([assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; (assume r' != 0 and inv; q = r / r'; r = r'; r' = r - q * r'; s = s'; s' = s - q * s'; t = t'; t' = t - q * t'; assert inv;) -> False] or [(assume pre; r = a; r' = b; s = 1; s' = 0; t = 0; t' = 1; assert inv; havoc r; havoc r'; havoc s; havoc s'; havoc t; havoc t'; assume(r' == 0) and inv)]) -> pos
```

In [ ]:
a = Symbol('a',INT)
b = Symbol('b',INT)
r = Symbol('r', INT)
r_prime = Symbol('r_prime', INT)
s = Symbol('s', INT)
s_prime = Symbol('s_prime', INT)
t = Symbol('t', INT)
t_prime = Symbol('t_prime', INT)
N = Symbol('N', INT)
q = Symbol('q', INT)
gcd = Symbol('gcd', FunctionType(INT,[INT,INT]))
aux = Symbol('aux', INT)

pre = And(GT(a, Int(0)), GT(b, Int(0)), GT(N, a), GT(N, b))

pos = And(Equals(r, gcd(a, b)), Equals(r_prime, Int(0)), Exists([s, t], Equals(gcd(a, b), Plus(Times(a, s), Times(b, t)))))

inv = And(GT(r, Int(0)), GT(N, r), Equals(r, Plus(Times(a, s), Times(b, t))))
ini = substitute(inv, {r:a, r_prime:b, s:Int(1), s_prime:Int(0), t:Int(0), t_prime:Int(1)})
pres = Implies(And(Not(Equals(r_prime, Int(0))), inv), substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(substitute(inv, {tP:t}), {sP:s}), {rP:r}), {t_prime: Minus(tP, Times(q, t_prime))}), {t: t_prime}), {s_prime: Minus(sP, Times(q, s_prime))}), {s: s_prime}), {r_prime: Minus(rP, Times(q, r_prime))}), {r: r_prime}), {q: Div(r, r_prime)}))
util = Implies(And(Equals(r_prime, Int(0)), inv), pos)
vc = Implies(pre, And(ini, ForAll([r,r_prime,s,s_prime,t,t_prime,rP,sP,tP], And(pres, util))))

prove(Implies(axioms, vc))